**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Mon Aug 22 10:07:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 2021_2022DS_Java_correction.gdoc
 7esba-1.gdoc
 7esba.gdoc
 blindverse.zip
 Bulletin-de-notes-2INFO.gsheet
 Cahier-Des-Charges-Template-Type.gdoc
 Classroom
'Colab Notebooks'
'currency dataset'
'Document sans titre (1).gdoc'
'Document sans titre (2).gdoc'
'Document sans titre.gdoc'
 email_3_9_2021.gdoc
'ENICARTHAGE_ING_INFO_2013_2016.zip (Unzipped Files)'
'Express differently (1).gdoc'
'Express differently.gdoc'
'Feuille de calcul sans titre.gsheet'
 message-1.gdoc
 message-7.gdoc
 message.gdoc
 moyennes-1-1-1.gsheet
'Présentation de prototypage (1).gslides'
'Présentation de prototypage.gslides'
'Projet Google Science Fair.gslides'
'Projet scientifique.gslides'
'Proposition de projet.gdoc'
 rapport_PSR.gdoc
 RévisRaja.gslides
 Sem1
' SGBD_Intro .gdoc'
 slide_1.pptx
 slide_2.pptx
 slide_3.pptx
 slide_5.pptx
 slides_4.pptx
 slides_6.pptx
 stock1.gdoc
'Suivi de projet.gsheet'
'summer_internship-resume (1).pdf'
 yolov3


**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15490, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 15490 (delta 10), reused 18 (delta 8), pack-reused 15460
Receiving objects: 100% (15490/15490), 14.10 MiB | 22.92 MiB/s, done.
Resolving deltas: 100% (10395/10395), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "barcode" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-08-22 10:09:29--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   101MB/s    in 1.5s    

2022-08-22 10:09:30 (101 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/20201211_100_rgb.png  
  inflating: data/obj/20201211_100_rgb.txt  
  inflating: data/obj/20201211_1012_rgb.png  
  inflating: data/obj/20201211_1012_rgb.txt  
  inflating: data/obj/20201211_1013_rgb.png  
  inflating: data/obj/20201211_1013_rgb.txt  
  inflating: data/obj/20201211_1015_rgb.png  
  inflating: data/obj/20201211_1015_rgb.txt  
  inflating: data/obj/20201211_1016_rgb.png  
  inflating: data/obj/20201211_1016_rgb.txt  
  inflating: data/obj/20201211_1017_rgb.png  
  inflating: data/obj/20201211_1017_rgb.txt  
  inflating: data/obj/20201211_1018_rgb.png  
  inflating: data/obj/20201211_1018_rgb.txt  
  inflating: data/obj/20201211_1019_rgb.png  
  inflating: data/obj/20201211_1019_rgb.txt  
  inflating: data/obj/20201211_101_rgb.png  
  inflating: data/obj/20201211_101_rgb.txt  
  inflating: data/obj/20201211_1020_rgb.png  
  inflating: data/obj/20201211_1020_rgb.txt  
  inflating: data/obj/20201211_1021_rgb.png  
 

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

1 0.419141 0.725000 0.194531 0.212162
['1', '0.419141', '0.725000', '0.194531', '0.212162']
0 data/obj/20201211_1056_rgb.txt
0 0.419141 0.725000 0.194531 0.212162
1 0.405859 0.430405 0.225781 0.252703
['1', '0.405859', '0.430405', '0.225781', '0.252703']
1 data/obj/20201211_2275_rgb.txt
0 0.405859 0.430405 0.225781 0.252703
0 0.471875 0.413514 0.321875 0.789189
['0', '0.471875', '0.413514', '0.321875', '0.789189']
2 data/obj/20201211_836_rgb.txt
0 0.471875 0.413514 0.321875 0.789189
1 0.487891 0.544595 0.125781 0.102703
['1', '0.487891', '0.544595', '0.125781', '0.102703']
3 data/obj/20201211_1548_rgb.txt
0 0.487891 0.544595 0.125781 0.102703
1 0.431641 0.529054 0.144531 0.233784
['1', '0.431641', '0.529054', '0.144531', '0.233784']
4 data/obj/20201211_5396_rgb.txt
0 0.431641 0.529054 0.144531 0.233784
1 0.447266 0.385135 0.189844 0.510811
['1', '0.447266', '0.385135', '0.189844', '0.510811']
5 data/obj/20201211_3628_rgb.txt
0 0.447266 0.385135 0.189844 0.510811
0 0.423047 0.496622 0.3

In [ ]:
import glob
images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/20201211_2266_rgb.png', 'data/obj/20201211_3697_rgb.png', 'data/obj/20201211_103_rgb.png', 'data/obj/20201211_137_rgb.png', 'data/obj/20201211_3513_rgb.png', 'data/obj/20201211_6128_rgb.png', 'data/obj/20201211_3635_rgb.png', 'data/obj/20201211_2375_rgb.png', 'data/obj/20201211_2036_rgb.png', 'data/obj/20201211_1054_rgb.png', 'data/obj/20201211_350_rgb.png', 'data/obj/20201211_1041_rgb.png', 'data/obj/20201211_3412_rgb.png', 'data/obj/20201211_1555_rgb.png', 'data/obj/20201211_2795_rgb.png', 'data/obj/20201211_3138_rgb.png', 'data/obj/20201211_1036_rgb.png', 'data/obj/20201211_1061_rgb.png', 'data/obj/20201211_2346_rgb.png', 'data/obj/20201211_6253_rgb.png', 'data/obj/20201211_3941_rgb.png', 'data/obj/20201211_3909_rgb.png', 'data/obj/20201211_1065_rgb.png', 'data/obj/20201211_1533_rgb.png', 'data/obj/20201211_100_rgb.png', 'data/obj/20201211_1072_rgb.png', 'data/obj/20201211_3693_rgb.png', 'data/obj/20201211_1558_rgb.png', 'data/obj/20201211_3039_rgb.png', 'data/obj/2020121

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 156360, rewritten_bbox = 0.001279 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.824272), count: 3, class_loss = 0.040297, iou_loss = 0.060869, total_loss = 0.101166 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.857801), count: 2, class_loss = 0.221261, iou_loss = 0.036731, total_loss = 0.257992 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 156365, rewritten_bbox = 0.001279 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.884426), count: 3, class_loss = 0.028260, iou_loss = 0.068309, t